<a href="https://colab.research.google.com/github/virf96/Chat-Bot/blob/master/DatosFaltantes_IndicadorAusencia_FeatureEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Indicador de Ausencia ==> Feature-engine


### Qué es Feature-engine?

Feature-engine es una librería de Python que hemos creado para este curso. 

- Feature-engine incluye todas las técnicas de ingeniería de variables descritas en este curso
- Feature-engine funciona como Scikit-learn, por lo tanto es fácil de aprender
- Feature-engine te permite implementar pasos de ingeniería de variables específicos para diferentes grupos de variables
- Feature-engine puede ser integrado con las pipelines de Scikit-learn pipeline permitiendo construir modelos fácilmente
** Feature-engine te permite diseñar y guardar un flujo de ingeniería de variables con procesos diseñados específicamente para diferentes grupos de variables.**

-------------------------------------------------------------------
Feature-engine puede ser instalado vía pip ==> pip install feature-engine

- Asegurate que haz instalado Feature-engine antes de correr este notebook

Para más detalle visita el [website the trainindata]( https://www.trainindata.com/feature-engine) 


## En este demo:

Vamos a usar **Feature Engine para generar un indicador de ausencia** usando los datos Ames House Price.


### Nota: 
* 'Imputer' se deriva del verbo en inglés 'to impute' que quiere decir sustituir o reemplazar. Imputer es el objeto que completa la sustitución, de ahí el nombre dado a la clase.



In [1]:
pip install feature_engine

     |████████████████████████████████| 9.5MB 7.3MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

#  feature engine
from feature_engine import missing_data_imputers as mdi

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# carguemos los datos con unas columnas seleccionadas

cols_to_use = [
    'BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt',
    'SalePrice'
]

data = pd.read_csv('/content/drive/My Drive/datasets/houseprice.csv', usecols=cols_to_use)
data.head()

,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,GarageYrBlt,SalePrice
0,65.0,196.0,Gd,NaN,2003.0,208500
1,80.0,0.0,Gd,TA,1976.0,181500
2,68.0,162.0,Gd,TA,2001.0,223500
3,60.0,0.0,TA,Gd,1998.0,140000
4,84.0,350.0,Gd,TA,2000.0,250000


In [6]:
#Veamos los datos faltantes
data.isnull().mean()

LotFrontage    0.177397
MasVnrArea     0.005479
BsmtQual       0.025342
FireplaceQu    0.472603
GarageYrBlt    0.055479
SalePrice      0.000000
dtype: float64

In [7]:
# separar datos en segmentos entrenamiento y prueba
#Esto porque solo debemos aprender la categoría a imputar o valor del set d entrenamiento
# primero, separemos el target (SalePrice) del resto de las variables

cols_to_use.remove('SalePrice')

X_train, X_test, y_train, y_test = train_test_split(data[cols_to_use],
                                                    data['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((1022, 5), (438, 5))

In [8]:
X_train.isnull().mean()

BsmtQual       0.023483
FireplaceQu    0.467710
LotFrontage    0.184932
MasVnrArea     0.004892
GarageYrBlt    0.052838
dtype: float64

## El Indicador de Ausencia de Feature-engine selecciona todas las por defecto

In [9]:
# llamemos el imputer de feature engine
# missing_only en el param how le indica al imputador
# que aprenda solo las variables que presentan datos faltantes

imputer = mdi.AddMissingIndicator(how='missing_only') # alternatively 'all'

In [10]:
# ajustemos el imputer al set de entrenamineto

imputer.fit(X_train)

AddMissingIndicator(how='missing_only', variables=None)

In [11]:
# cuidado, a diferencia de otros imputadores, en el attributo variables
# el imputador guarda los valores entrados por el usuario, en este caso
# None
imputer.variables

In [12]:
# vemos que el imputer encontró las variables categóricas 
# y numéricas

# en este attributo, el imputador selecciona aquellas variables que han
# mostrado NA en el set de entrenamiento

imputer.variables_

['BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [13]:
# feature engine devuelve un dataframe
# con las variables o features adicionales

# no hay necesidad de concatenar!!

tmp = imputer.transform(X_train)
tmp.head()

,BsmtQual,FireplaceQu,LotFrontage,MasVnrArea,GarageYrBlt,BsmtQual_na,FireplaceQu_na,LotFrontage_na,MasVnrArea_na,GarageYrBlt_na
64,Gd,NaN,NaN,573.0,1998.0,0,1,1,0,0
682,Gd,Gd,NaN,0.0,1996.0,0,0,1,0,0
960,TA,NaN,50.0,0.0,NaN,0,1,0,0,1
1384,TA,NaN,60.0,0.0,1939.0,0,1,0,0,0
1100,TA,NaN,60.0,0.0,1930.0,0,1,0,0,0


In [14]:
# revisemos que ya no tenemos valores nulos

tmp.isnull().mean()

BsmtQual          0.023483
FireplaceQu       0.467710
LotFrontage       0.184932
MasVnrArea        0.004892
GarageYrBlt       0.052838
BsmtQual_na       0.000000
FireplaceQu_na    0.000000
LotFrontage_na    0.000000
MasVnrArea_na     0.000000
GarageYrBlt_na    0.000000
dtype: float64

Vemos que si los tenemos, recuerda que debemos imputar las variables originales con otra técnica de imputación

## Feature-engine  te permite especificar grupos de variables fácilmente

In [15]:
# hagamos la sustitución pero esta vez seleccionamos algunas
# de las variables

imputer = mdi.AddMissingIndicator(variables=['BsmtQual', 'FireplaceQu', 'LotFrontage'])

imputer.fit(X_train)

AddMissingIndicator(how='missing_only',
                    variables=['BsmtQual', 'FireplaceQu', 'LotFrontage'])

In [16]:
# ahora el imputer usa solo las variables que indicamos

imputer.variables

['BsmtQual', 'FireplaceQu', 'LotFrontage']

In [17]:
imputer.variables_

['BsmtQual', 'FireplaceQu', 'LotFrontage']

In [18]:
# feature-engine devuelve un dataframe
# con las variables o features adicionales

# no hay necesidad de contatenar!!

tmp = imputer.transform(X_train)

tmp.head()

,BsmtQual,FireplaceQu,LotFrontage,MasVnrArea,GarageYrBlt,BsmtQual_na,FireplaceQu_na,LotFrontage_na
64,Gd,NaN,NaN,573.0,1998.0,0,1,1
682,Gd,Gd,NaN,0.0,1996.0,0,0,1
960,TA,NaN,50.0,0.0,NaN,0,1,0
1384,TA,NaN,60.0,0.0,1939.0,0,1,0
1100,TA,NaN,60.0,0.0,1930.0,0,1,0


## Feature-engine puede ser usado con los flujos de Scikit-learn (pipeline)

In [19]:
# revisemos el porcentaje de valores nulos por variable

X_train.isnull().mean()

BsmtQual       0.023483
FireplaceQu    0.467710
LotFrontage    0.184932
MasVnrArea     0.004892
GarageYrBlt    0.052838
dtype: float64

In [20]:
X_train.head()

,BsmtQual,FireplaceQu,LotFrontage,MasVnrArea,GarageYrBlt
64,Gd,NaN,NaN,573.0,1998.0
682,Gd,Gd,NaN,0.0,1996.0
960,TA,NaN,50.0,0.0,NaN
1384,TA,NaN,60.0,0.0,1939.0
1100,TA,NaN,60.0,0.0,1930.0


Estos son los pasos que vamos a poner en un flujo de trabajo o pipeline

- Añadir un Indicador de Ausencia a todas las variables
- Sustitución por la mediana para las variables numéricas
- Sustitución de la categoría faltante en las variables categóricas

In [21]:
pipe = Pipeline([
    ('missing_ind', mdi.AddMissingIndicator()),
    ('imputer_missing', mdi.CategoricalVariableImputer(variables=['FireplaceQu', 'BsmtQual'])),
    ('imputer_median', mdi.MeanMedianImputer(imputation_method = 'median',
                                             variables=['LotFrontage', 'MasVnrArea', 'GarageYrBlt'])),
])

In [23]:
# ajustar el pipeline al set de entrenamiento

pipe.fit(X_train)

Pipeline(memory=None,
         steps=[('missing_ind',
                 AddMissingIndicator(how='missing_only', variables=None)),
                ('imputer_missing',
                 CategoricalVariableImputer(fill_value='Missing',
                                            imputation_method='missing',
                                            return_object=False,
                                            variables=['FireplaceQu',
                                                       'BsmtQual'])),
                ('imputer_median',
                 MeanMedianImputer(imputation_method='median',
                                   variables=['LotFrontage', 'MasVnrArea',
                                              'GarageYrBlt']))],
         verbose=False)

In [24]:
# inspeccionemos cada paso por separado

pipe.named_steps['missing_ind'].variables_

['BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [25]:
pipe.named_steps['imputer_missing'].variables

['FireplaceQu', 'BsmtQual']

In [26]:
pipe.named_steps['imputer_median'].imputer_dict_

{'GarageYrBlt': 1979.0, 'LotFrontage': 69.0, 'MasVnrArea': 0.0}

In [27]:
# ahora transformemos los datos con la pipeline

# Este flujo de trabajo permitirá:
#- añadir los indicadores de ausencia
#- llenar los valores nulos en las variables originales
# preparar los datos para poder ser usados en un modelo de ML


tmp = pipe.transform(X_train)

# revisemos que ya no tenemos valores nulos
tmp.isnull().mean()

BsmtQual          0.0
FireplaceQu       0.0
LotFrontage       0.0
MasVnrArea        0.0
GarageYrBlt       0.0
BsmtQual_na       0.0
FireplaceQu_na    0.0
LotFrontage_na    0.0
MasVnrArea_na     0.0
GarageYrBlt_na    0.0
dtype: float64

In [28]:
tmp.shape

(1022, 10)

Más información aquí:
https://feature-engine.readthedocs.io/en/latest/imputers/AddMissingIndicator.html